# 네이버 뉴스에서 `비트코인` 관련 뉴스 수집하기 (`selenium` 사용)

시작하기 전.. Anaconda prompt에서 아래와 같이 입력하여 `selenium`을 설치하자.

```
conda install selenium
```

# `requests` 패키지 대신 `selenium`으로 네이버 뉴스 수집하기

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()

# 키워드, URL
keyword = '비트코인'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

# url 접속
driver.get(url)
driver.implicitly_wait(2) # 2초 안에 웹페이지를 load하면 바로 넘어가거나, 2초를 기다림

elements = driver.find_elements('css selector', 'div.news_wrap.api_ani_send')
for x in elements:
    # title, url
    #sp_nws1 > div.news_wrap.api_ani_send > div > div.news_contents > a.news_tit
    title = x.find_element("css selector", 'div > div.news_contents > a.news_tit').text
    url = x.find_element("css selector", 'div > div.news_contents > a.news_tit').get_attribute("href")
    # publisher
    # #sp_nws1 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a.info.press
    pub = x.find_element("css selector", 'div > div.news_info > div.info_group > a.info.press').text.replace('언론사 선정', '')

    print(title)
    print(url)
    print(pub)
    print()

# don't forget to close the driver
driver.close()

# 동적 스크롤 도입

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# driver
driver = webdriver.Chrome()

# 키워드, URL
keyword = '비트코인'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

# get page
driver.get(url)
driver.implicitly_wait(2) # 2초 안에 웹페이지를 load하면 바로 넘어가거나, 2초를 기다림

# scroller
actions = driver.find_element('css selector', 'body')

actions.send_keys(Keys.END)
time.sleep(2)

actions.send_keys(Keys.END)
time.sleep(2)

actions.send_keys(Keys.END)
time.sleep(2)

# 스크롤 수만큼 페이지 로딩 후 뉴스 스크랩

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


# 키워드, URL
keyword = '비트코인'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

# driver
driver = webdriver.Chrome()

# get page
driver.get(url)
driver.implicitly_wait(2) # 2초 안에 웹페이지를 load하면 바로 넘어가거나, 2초를 기다림

# scroller
actions = driver.find_element('css selector', 'body')

def crawling(num_scroll):
    
    for _ in range(num_scroll):
        actions.send_keys(Keys.END)
        time.sleep(2)
    
    titles, presses, links = [], [], []
    title_selector = 'div.news_wrap.api_ani_send > div > div.news_contents > a.news_tit'
    press_selector = 'div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a.info.press'

    for i in driver.find_elements('css selector', title_selector):
        presses.append(i.text.replace('언론사 선정', ''))
    
    for i in driver.find_elements('css selector', title_selector):
        titles.append(i.text)
        links.append(i.get_attribute("href"))
    
    return titles, presses, links


num_scroll = 2
titles, presses, links = crawling(num_scroll)

print("가져온 뉴스 수: ", len(titles))
print('')

for title, press, link in zip(titles, presses, links):
    print("제목: ", title)
    print("언론사: ", press)
    print("주소: ", link)
    print()

# don't forget to close the driver!
driver.close()

In [ ]:
import pandas as pd

# 뉴스 제목 데이터프레임 생성
df = pd.DataFrame({'url': links,
                   'press': presses,
                   'title': titles})
df

# 뉴스 내용까지 수집하기

아래와 같이 `newspaper3k` 패키지를 설치하자
```
pip install newspaper3k
```

- documentation: https://newspaper.readthedocs.io/en/latest/

In [ ]:
from newspaper import Article

texts = []

for link in links:
    # 언어를 한국어로 설정하고 URL을 전달해 Article 클래스의 객체 생성
    article = Article(link, language='ko')
    # 지정된 웹 페이지를 다운로드
    article.download()
    # 다운로드한 웹 페이지를 분석하고 필요한 정보를 추출
    article.parse()
    
    # 기사 제목 출력
    print('기사 제목 :', article.title)
    # 기사 내용 출력 
    print('기사 내용 :', article.text[:20])
    print('')

    texts.append(article.text)
    
    time.sleep(1)

In [ ]:
df['text'] = texts

df